# Multi Layer Perceptron NLP Model by Bengio, Manual Back-propagation

This is an implementation of a character level NLP model, motivated by [Bengio et al.](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) paper, based on Andrej Karpathy's [Makemore](https://www.youtube.com/watch?v=TCH_1BHY58I&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=3) lectures.

<br>
<br>
<figure align="center">
    <img src="./images/bengio_2003_neural_architecture.png" width="500">
    <figcaption>Bengio et al.'s neural model.</figcaption>
</figure>

, where $w_t$ is the t-th word, and $w_i^j = \left(w_i, w_{i+1}, \ldots, w_{j-1}. w_j\right)$ is a sequence of words from $i$ to $j$. Since in this example we will teach a neural network to generate names based on examples, instead of using words, we use characters. This version uses batch normalization in order to deal with the gradient vanishing problem.

## Manual Back-propagation

This version shows how the backpropagation can be done manually without using PyTorch's autograd.

## Input to the Model

The input to the model is a sequence of n-characters (we use 3 in the code). These characters are converted into a 2D embedding space using a look-up-table C. LUT is part of the nn model, so the weights for the LUT are learned. The rationale for using a learned embedding space is for the model to be able to generalize better, especially when it comes across input that is has not seen previously.

## Output from the Model 

For each combination of 3 input characters, the model outputs a probability distribution for the 27 letters from the alphabet. Using the probability distribution we can sample letters.

## Loss Function

We need a way of measuring how well the model is doing so that the deep-learning model can learn based on the given examples. In order to do so, we need a loss function. We start by defining the likelihood function. Likelihood function is the joint probability of the observed data viewed as a function of the parameters of a statistical model (source: [Wikipedia](https://en.wikipedia.org/wiki/Likelihood_function)). In statistics, maximum likelihood estimation is a method of estimating the parameters of an assumed probability distribution, given observed data. For a set of independent and identically distributed points $X = \{x_1, x_2,..., x_n\}$, the likelihood is defined as follows:

$$\mathcal{L}(\theta|x)=p(X|\theta)=\prod_{n=1}^{N}p(x_n|\theta)$$

$\mathcal{L}(\theta|x)$ is the likelihood function, $\theta$ are the model parameters and $x$ are the observations. We want to maximize the likelihood of the data with respect to the parameters. In other words, we want to maximize the likelihood, by tweaking the parameters $\theta$, that the model produces the observed data. Since we have many bigrams, we want to maximize the likelihood over all of them, which leads to a product over all of the probabilities. Since the probability for each individual observation is between $[0...1]$, multiplying these together leads to a small number and numerical instabilities. Therefore, we want to maximize the log-likelihood. Since $log(a \cdot b)=log(a)+log(b)$, we get:

$$log\left(\mathcal{L}(\theta|x)\right)=\sum_{n=1}^{N}log\left(p(x_n|\theta)\right)$$

The way we train deep-learning networks is by minizing a loss function. Maximizing the log-likelihood is equivalent to minimizing the negative log-likelihood, and in order for the results to be comparable regardless the number of data points we are using, we normalize the negative log-likelihood by dividing it with the number of data points we have. Therefore, our loss function is:

$$-\dfrac{1}{N}\sum_{n=1}^{N}log\left(p(x_n|\theta)\right)$$

In [1]:
import urllib.request
import torch
import torch.nn as nn
from torch import functional as F
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# Url to the file containing over 30k names
url_source = "https://raw.githubusercontent.com/karpathy/makemore/master/names.txt"

text = str('')

# Read to a variable, line by line
for line in urllib.request.urlopen(url_source):
    text += line.decode('utf-8')

In [3]:
# Split the file to lines and show the first 10 lines
words = text.splitlines()
print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [4]:
# Create the dictionaries to map characters to integers and vice versa
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(f"{stoi=}")
print(f"{itos=}")

stoi={'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
itos={1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


## Training the Model

In [5]:
block_size = 3 # how many characters we use in order to predict the next one

def build_dataset(words_in, block_size_in=3):
    
    X, Y = [], []

    for w in words_in:
        # This creates a list, filled with zeros:, that has the length defined by the variable block_size
        context = [0] * block_size_in
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size)
Xval, Yval = build_dataset(words[n1:n2], block_size)
Xte, Yte = build_dataset(words[n2:], block_size)

In [6]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}")

In [7]:
embedding_size = 10
hidden_layer_size = 64

# As a good starting point, we would like the probability distribution to be roughly 1/27. This simply means
# that each character has the same likelihood of being sampled.

# We use Kaiming-normal for initialisation (https://pytorch.org/docs/stable/nn.init.html)
# Gain for tanh is 5/3 (https://pytorch.org/docs/stable/nn.init.html)

def kaiming(fan_in):
    return (5/3) / (fan_in**0.5)

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, embedding_size), generator=g)
# Layer 1
W1 = torch.randn((embedding_size*block_size, hidden_layer_size), generator=g) * kaiming(embedding_size*block_size)
b1 = torch.randn(hidden_layer_size, generator=g) * 0.1
# Layer 2
W2 = torch.randn((hidden_layer_size, 27), generator=g) * 0.1
b2 = torch.randn(27, generator=g) * 0.1
# Batch normalization
bngain = torch.randn((1, hidden_layer_size))*0.1 + 1.0
bnbias = torch.randn((1, hidden_layer_size))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]

for p in parameters:
    p.requires_grad = True

In [8]:
losses = np.array([])
batch_size = 32
n = batch_size

# Create a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [9]:
#--- FORWARD PASS ---
emb = C[Xb]
embcat = emb.view(-1, W1.shape[0])

# Linear layer
z_pre_n = embcat @ W1 + b1

# Batch normalization layer
#---------------------------------------------------------------------------
bnmean = 1/n * z_pre_n.sum(0, keepdim=True)
bndiff = z_pre_n - bnmean
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
z_pre_act = bngain*bnraw + bnbias
#---------------------------------------------------------------------------

# Non-linearity/activation
a1 = torch.tanh(z_pre_act) # hidden layer
logits = a1 @ W2 + b2 # output layer

# Cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

#--- BACKWARD PASS ---
# Reset the gradients
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, a1, z_pre_act, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, z_pre_n, bnmean,
          embcat, emb]:
    t.retain_grad()
loss.backward()

In [10]:
logprobs.shape, Yb.shape

(torch.Size([32, 27]), torch.Size([32]))

In [18]:
# Exercise 1: backprob through the whole thing manually
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# 1. dloss/dlogprobs, where loss = -logprobs[range(n), Yb].mean()
# dlogprobs will hold the derivative of the loss with respect to all the elements in the logprobs tensor.
# Suppose we have the following:
# loss = -1/3*(a + b + c), then
# dloss/da = -1/3
# dloss/db = -1/3
#, so in more general case where we have:
# loss = -1/n*sum[i=1..n]f_i
# dloss/df_i = -1/n
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n

# 2. dloss/dprobs
# dprobs will hold the derivative of the dlogprobs with respect to all the elements in the probs tensor
# Chain rule: h(x) = f(g(x)), then h'(x) = f'(g(x))g'(x)
# f'(g(x)) = dlogprobs
# g'(x) = (1.0/probs)
dprobs = dlogprobs * (1.0/probs)

# 3. dloss/dcounts_sum_inv
# probs = counts * counts_sum_inv, where counts_sum_inv gets 'replicated' to the correct shape
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
# dprobs/dcounts
dcounts = counts_sum_inv * dprobs

# 4. dloss/dcounts_sum
dcounts_sum = -1 * counts_sum**-2 * dcounts_sum_inv

# 5. dcounts
# This is the second branch of the dcounts, first is from dprobs/dcounts
# This is a summation:
# a11 a12 a13  --->  b1 (= a11 + a12 + a13)
# a21 a22 a23  --->  b2 (= a21 + a22 + a23)
# a31 a32 a33  --->  b3 (= a31 + a32 + a33)
#, how to b:s depend on a:s?
#          1 1 1
# dA/db1 = 0 0 0
#          0 0 0
#
dcounts += torch.ones_like(counts) * dcounts_sum

# 6. dnorm_logits
dnorm_logits = counts * dcounts

# ...rest is still missing...

cmp('logprobs', dlogprobs, logprobs)
cmp('dprobs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('dnorm_logits', dnorm_logits, norm_logits)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
dprobs          | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
dnorm_logits    | exact: True  | approximate: True  | maxdiff: 0.0
